<a href="https://colab.research.google.com/github/jiiwon129/ESAA/blob/main/OB/%EC%88%98%EC%83%81%EC%9E%91%20%EB%A6%AC%EB%B7%B0/ESAA_OB_Award_Review2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2025 전력사용량 예측 AI 경진대회**

- reference link1: https://dacon.io/competitions/official/236531/overview/description

- reference link2: https://dacon.io/competitions/official/236531/codeshare/12766?page=1&dtype=recent

## **주제**

건물의 전력사용량 예측 AI 모델 개발
- 건물의 전력사용량 데이터, 기상데이터 등 시공간 정보를 활용하여 전력 사용량을 예측하는 AI 모델을 개발

### **평가 지표**

SMAPE (Symmetric Mean Absolute Percentage Error)

$\text{SMAPE} = \frac{100\%}{n} \sum_{t=1}^{n} \frac{2 \cdot |\hat{y}_t - y_t|}{|\hat{y}_t| + |y_t|}$

```
def smape(gt, preds):
    gt= np.array(gt)
    preds = np.array(preds)
    v = 2 * abs(preds - gt) / (abs(preds) + abs(gt))
    score = np.mean(v) * 100
    return score
```

```
def custom_smape(preds, dtrain):
    labels = dtrain.get_label()
    return 'custom_smape', np.mean(2 * abs(preds - labels) / (abs(preds) + abs(labels))) * 100
```

---
SMAPE
- 오차를 백분율로 나타낸다는 점에서 MAPE(Mean Absolute Percentage Error)와 유사

SMAPE를 사용하는 이유
1. **대칭성(Symmetry)**: SMAPE는 분모에 실제값과 예측값의 평균을 사용하기 때문에, 오차가 실제값에 비례하여 과도하게 계산되는 것을 방지
- 즉, 예측값이 실제값보다 크거나 작을 때의 오차를 균형 있게 평가

2. **제로 값 문제 해결**: MAPE는 실제값이 0일 경우 분모가 0이 되어 오차가 무한대가 되는 문제가 발생하지만, SMAPE는 분모에 예측값까지 포함하므로 이러한 문제가 발생하지 않음

주로 시계열 예측에 사용되는 평가 지표
- 특히, **실제값이 0에 가까운 경우**나 **예측값이 실제값보다 크거나 작은 오차를 공평하게 비교해야하는 상황**에서 유용

## **데이터 설명**

Dataset Info.

❶10개 유형* 100개 건물의 전력소비량 데이터(1시간주기, ‘24.6.1～8.31), ❷기상데이터**, ❸건물개요(면적, 태양광·ESS 용량 등)

* 공공, 학교, 백화점, 병원, 아파트, 호텔 등

** 기온, 강수량, 풍속, 습도, 일조, 일사

1. train.csv

85일 분량의 데이터
train 데이터 : 100개 건물들의 2024년 06월 01일부터 2024년 08월 24일까지의 데이터
일시별 기온, 강수량, 풍속, 습도, 일조, 일사 정보 포함
전력사용량(kWh) 포함


2. building_info.csv

100개 건물 정보
건물 번호, 건물 유형, 연면적, 냉방 면적, 태양광 용량, ESS 저장 용량, PCS 용량


3. test.csv

test 데이터 : 100개 건물들의 2024년 08월 25일부터 2024년 08월 31일까지의 데이터
일시별 기온, 강수량, 풍속, 습도의 예보 정보


4. sample_submission.csv

제출을 위한 양식
100개 건물들의 2024년 08월 25일부터 2024년 08월 31일까지의 전력사용량(kWh)을 예측
num_date_time은 건물번호와 시간으로 구성된 ID
해당 ID에 맞춰 전력사용량 예측값을 answer 컬럼에 기입

## **데이터 전처리**

### 1. Data Cleaning

- power_consumption == 0인 데이터 행 전체 제거
- 특정 건물 및 날짜의 휴일 데이터 제거

### 2. Feature Engineering

- 시간 관련 변수 생성
  1) 기본 변수: date_time -> hour, day, month, day_of_week 추출

```
train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')

# Datetime
  train['hour'] = train['date_time'].dt.hour
  train['day'] = train['date_time'].dt.day
  train['month'] = train['date_time'].dt.month
  train['day_of_week'] = train['date_time'].dt.dayofweek
```
```
# Calculate 'day_temperature'
    def calculate_day_values(dataframe, target_column, output_column, aggregation_func):
        result_dict = {}

        grouped_temp = dataframe.groupby(['building_number', 'month', 'day'])[target_column].agg(aggregation_func)

        for (building, month, day), value in grouped_temp.items():
            result_dict.setdefault(building, {}).setdefault(month, {})[day] = value

        dataframe[output_column] = [
            result_dict.get(row['building_number'], {}).get(row['month'], {}).get(row['day'], None)
            for _, row in dataframe.iterrows()
        ]

    train['day_max_temperature'] = 0.0
    train['day_mean_temperature'] = 0.0

    calculate_day_values(train, 'temperature', 'day_max_temperature', 'max')
    calculate_day_values(train, 'temperature', 'day_mean_temperature', 'mean')
    calculate_day_values(train, 'temperature', 'day_min_temperature', 'min')

    train['day_temperature_range'] = train['day_max_temperature'] - train['day_min_temperature']
```

  2) 주기성 변수: 시간의 연속성을 모델에 효과적으로 학습시키기 위해 삼각함수 변환 적용
  - sin_hour, cos_hour (한 시간 주기)

```
train['sin_hour'] = np.sin(2 * np.pi * train['hour']/23.0)
train['cos_hour'] = np.cos(2 * np.pi * train['hour']/23.0)
```
  - sin_date, cos_date (하루 주기)

```
train['sin_date'] = -np.sin(2 * np.pi * (train['month']+train['day']/31)/12)
train['cos_date'] = -np.cos(2 * np.pi * (train['month']+train['day']/31)/12)
```

  - sin_dayofweek, cos_dayofweek (일주일 주기)

```
train['sin_dayofweek'] = -np.sin(2 * np.pi * (train['day_of_week']+1)/7.0)
train['cos_dayofweek'] = -np.cos(2 * np.pi * (train['day_of_week']+1)/7.0)
```
  - sin_month, cos_month (일년 주기)

```
train['sin_month'] = -np.sin(2 * np.pi * train['month']/12.0)
train['cos_month'] = -np.cos(2 * np.pi * train['month']/12.0)
```

- 공휴일 변수
  - 주말(day_of_week >= 5) 또는 지정된 공휴일을 hoilday=1로 처리하여 평일과 구분

```
holi_weekday = ['2024-06-06', '2024-08-15']

train['holiday'] = np.where((train.day_of_week >= 5) | (train.date_time.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)
```

- 기상 및 전력 패턴 파생변수
  - 기상 복합 지수
    1. CDH(Cooling Degree Hour)
    - cumsum(온도-26)을 이용해 26도를 초과하는 냉방 부하 누적량을 변수화

```
def CDH(xs):
        cumsum = np.cumsum(xs - 26)
        return np.concatenate((cumsum[:11], cumsum[11:] - cumsum[:-11])) # 계산된 누적 합을 바탕으로 1시간 단위의 CDH를 반환

def calculate_and_add_cdh(dataframe):
        cdhs = []
        for i in range(1, 101):
            temp = dataframe[dataframe['building_number'] == i]['temperature'].values
            cdh = CDH(temp)
            cdhs.append(cdh)
        return np.concatenate(cdhs)

train['CDH'] = calculate_and_add_cdh(train)
```

2. THI(Temperature-Humidity Index)
      - 온도와 습도를 조합한 불쾌지수

```
train['THI'] = 9/5*train['temperature'] - 0.55*(1-train['humidity']/100)*(9/5*train['humidity']-26)+32
```

3. WCT(Wind Chill Temperature)
      - 온도와 풍속을 조합한 체감온도

```
train['WCT'] = 13.12 + 0.6125*train['temperature'] - 11.37*(train['windspeed']**0.16) + 0.3965*(train['windspeed']**0.16)*train['temperature']
```

- 통계 기반 전력 패턴 변수
  - 과거 데이터의 패턴을 변수로 주입
    1. day_hour_mean/std
      - 건물별, 요일별, 시간별 평균/표준편차 전력량

```
power_mean = pd.pivot_table(train, values='power_consumption', index=['building_number', 'hour', 'day_of_week'], aggfunc=np.mean).reset_index()
    power_mean.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_mean']

power_std = pd.pivot_table(train, values='power_consumption', index=['building_number', 'hour', 'day_of_week'], aggfunc=np.std).reset_index()
    power_std.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_std']

train = train.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')

train = train.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')
```

2. hour_mean/std
- 건물별, 시간별 평균/표준편차 전력량

```
 power_hour_mean = pd.pivot_table(train, values='power_consumption', index=['building_number', 'hour'], aggfunc=np.mean).reset_index()
    power_hour_mean.columns = ['building_number', 'hour', 'hour_mean']

power_hour_std = pd.pivot_table(train, values='power_consumption', index=['building_number', 'hour'], aggfunc=np.std).reset_index()
    power_hour_std.columns = ['building_number', 'hour', 'hour_std']

train = train.merge(power_hour_mean, on=['building_number', 'hour'], how='left')

train = train.merge(power_hour_std, on=['building_number', 'hour'], how='left')
```

- 여름 기간 특화 변수(실험적 접근)

```
 if summer == True:
        def summer_cos(date):
            # 2024년 6월 1일부터 2024년 9월 14일까지를 여름 기간으로 정의
            start_date = datetime.strptime("2024-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")
            end_date = datetime.strptime("2024-09-14 00:00:00", "%Y-%m-%d %H:%M:%S")

            # start_date와 end_date 사이의 총 시간을 초(second) 단위로 계산
            period = (end_date - start_date).total_seconds()

            # 현재 날짜(date)가 여름 주기에서 어느 시점에 해당하는지를 라디안(radian) 값으로 변환
            return math.cos(2 * math.pi * (date - start_date).total_seconds() / period)

        def summer_sin(date):
            start_date = datetime.strptime("2024-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")
            end_date = datetime.strptime("2024-09-14 00:00:00", "%Y-%m-%d %H:%M:%S")

            period = (end_date - start_date).total_seconds()

            return math.sin(2 * math.pi * (date - start_date).total_seconds() / period)

        train['summer_cos'] = train['date_time'].apply(summer_cos)
        train['summer_sin'] = train['date_time'].apply(summer_sin)

        test['summer_cos'] = test['date_time'].apply(summer_cos)
        test['summer_sin'] = test['date_time'].apply(summer_sin)
```



## **모델링**

### XGBoost Regressor

선정 이유
- 데이터 특성상 범주형 변수보다 수치형 변수가 많음.

### Ensemble

#### 1. 전체 모델
- 모든 데이터를 하나의 모델로 학습하여 일반적인 패턴 학습

```
answer_df_global = pd.DataFrame(index=test_X.index, columns=["answer"], dtype=float)
pred_df_global = pd.DataFrame(index=X.index, columns=["pred"], dtype=float)

x_global = pd.get_dummies(X.copy(), columns=["building_type"], drop_first=False)
xt_global = pd.get_dummies(test_X.copy(), columns=["building_type"], drop_first=False)

x_global = pd.get_dummies(x_global, columns=["building_number"], drop_first=False)
xt_global = pd.get_dummies(xt_global, columns=["building_number"], drop_first=False)

drop_cols_global = []

x_global = x_global.drop(columns=drop_cols_global, errors='ignore')
xt_global = xt_global.drop(columns=drop_cols_global, errors='ignore')

xt_global = xt_global.reindex(columns=x_global.columns, fill_value=0)

y_global = Y['power_consumption'].copy()

preds_valid_global = pd.Series(index=y_global.index, dtype=float)
preds_test_global = []

x_values_global = x_global.values
y_values_global = y_global.values

fold_scores_global = []
for fold, (tr_idx, va_idx) in enumerate(kf.split(x_values_global), 1):
    X_tr, X_va = x_values_global[tr_idx], x_values_global[va_idx]
    y_tr, y_va = y_values_global[tr_idx], y_values_global[va_idx]

    y_tr_log = np.log(y_tr)
    y_va_log = np.log(y_va)

    model_global = XGBRegressor(
            learning_rate     = 0.05,
            n_estimators      = 5000,
            max_depth         = 10,
            subsample         = 0.7,
            colsample_bytree  = 0.5,
            min_child_weight  = 3,
            random_state      = RANDOM_SEED,
            objective         = weighted_mse(3),
            tree_method       = "gpu_hist",
            gpu_id            = 0,
            early_stopping_rounds = 100,
        )

    model_global.fit(
            X_tr, y_tr_log,
            eval_set=[(X_va, y_va_log)],
            eval_metric=custom_smape,
            verbose=False,
        )

    va_pred = np.exp(model_global.predict(X_va))
    preds_valid_global.iloc[va_idx] = va_pred

    fold_smape = smape(y_va, va_pred)
    fold_scores_global.append(fold_smape)

    preds_test_global.append(np.exp(model_global.predict(xt_global.values)))

pred_df_global.loc[preds_valid_global.index, "pred"] = preds_valid_global
answer_df_global.loc[xt_global.index, "answer"] = np.mean(preds_test_global, axis=0)

print(f"Global Model : XGB SMAPE = {np.mean(fold_scores_global):.4f}")

total_smape_global = smape(
    Y.sort_index()["power_consumption"].values,
    pred_df_global.sort_index()["pred"].values
)
print(f"Total SMAPE (Global) = {total_smape_global:.4f}")
```

---

#### 2. 건물 유형별 모델
- 10개 건물 유형으로 데이터를 분리하여 유형별 공통 패턴 학습

```
# No Summer Feature

X = train.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity',
                'power_consumption','rainfall', 'sunshine', 'solar_radiation',
                'hour','day','month','day_of_week','date_time'],axis =1 )

Y = train[['building_type','power_consumption']]

test_X = test.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity','rainfall',
                   'hour','month','day_of_week','day','date_time'], axis=1)
```

```
type_list = []
for value in train.building_type.values:
    if value not in type_list:
        type_list.append(value)

max_depth_dict = {
    'Other Buildings': 10,
    'Public': 10,
    'University': 8,
    'IDC': 6,
    'Department Store': 8,
    'Hospital': 8,
    'Commercial': 10,
    'Apartment': 6,
    'Research Institute': 10,
    'Hotel': 10
}
```

modeling

```
# 건물 타입별 모델
type_list = X["building_type"].unique()

answer_df = pd.DataFrame(index=test_X.index, columns=["answer"], dtype=float)
pred_df   = pd.DataFrame(index=X.index,       columns=["pred"],   dtype=float)

kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=RANDOM_SEED)

for btype in type_list:
    # 현재 반복문의 건물 유형(btype)에 해당하는 훈련(x, y) 및 테스트(xt) 데이터를 분리
    x  = X  [X['building_type'] == btype].copy()
    y  = Y  [Y['building_type'] == btype]['power_consumption'].copy()
    xt = test_X[test_X['building_type'] == btype].copy()

    # 원-핫 인코딩
    x  = pd.get_dummies(x,  columns=["building_number"], drop_first=False)
    xt = pd.get_dummies(xt, columns=["building_number"], drop_first=False)

    # 훈련 데이터에만 존재하는 건물 번호 컬럼이 테스트 데이터에도 동일하게 존재하도록 맞춤
    xt = xt.reindex(columns=x.columns, fill_value=0)

    # 모델 학습에 직접 사용되지 않는 building_type 컬럼을 제거
    drop_cols = ["building_type"]
    x  = x .drop(columns=drop_cols)
    xt = xt.drop(columns=drop_cols)

    preds_valid = pd.Series(index=y.index, dtype=float)
    preds_test  = []

    x_values = x.values
    y_values = y.values

    fold_scores = []
    for fold, (tr_idx, va_idx) in enumerate(kf.split(x_values), 1):
        X_tr, X_va = x_values[tr_idx], x_values[va_idx]
        y_tr, y_va = y_values[tr_idx], y_values[va_idx]

        # 타겟 변수(y)인 전력 소비량의 스케일이 크고 편중되어 있을 수 있으므로, np.log 변환을 적용하여 모델 학습을 안정화
        y_tr_log = np.log(y_tr)
        y_va_log = np.log(y_va)

        model = XGBRegressor(
            learning_rate     = 0.05,
            n_estimators      = 5000,
            max_depth         = max_depth_dict[btype],
            subsample         = 0.7,
            colsample_bytree  = 0.5,
            min_child_weight  = 3,
            random_state      = RANDOM_SEED,
            objective         = weighted_mse(3),
            tree_method       = "gpu_hist",
            gpu_id            = 0,
            early_stopping_rounds = 100,
        )

        model.fit(
            X_tr, y_tr_log,
            eval_set=[(X_va, y_va_log)],
            eval_metric=custom_smape,
            verbose=False,
        )

        # 로그 변환된 예측값을 다시 np.exp를 사용하여 원래 스케일로 복구
        va_pred = np.exp(model.predict(X_va))

        # 검증 세트의 예측값을 preds_valid에 저장
        preds_valid.iloc[va_idx] = va_pred

        # 각 폴드의 SMAPE 점수를 계산하여 저장
        fold_smape = smape(y_va, va_pred)
        fold_scores.append(fold_smape)

        # 테스트 데이터(xt)에 대한 예측값을 계산하여 preds_test 리스트에 저장
        preds_test.append(np.exp(model.predict(xt.values)))

    # 건물 유형별 교차 검증 예측값(preds_valid)을 pred_df의 올바른 위치에 할당
    pred_df.loc[preds_valid.index, "pred"] = preds_valid

    # 각 폴드의 테스트 예측값들을 평균 내어 최종 예측값으로 사용하고, answer_df에 저장
    answer_df.loc[xt.index, "answer"] = np.mean(preds_test, axis=0)

    print(f"Building type = {btype} : XGB SMAPE = {np.mean(fold_scores):.4f}")

total_smape = smape(
    Y.sort_index()["power_consumption"].values,
    pred_df.sort_index()["pred"].values
)
print(f"Total SMAPE = {total_smape:.4f}")
```
---

#### 3. 건물별 모델
- 100개 건물에 대해 각각의 모델을 생성하여 건물 고유의 미세한 특성 학습

```
Y = train[['building_number','power_consumption']]

answer_df_by_building = pd.DataFrame(index=test_X.index, columns=["answer"], dtype=float)
pred_df_by_building = pd.DataFrame(index=X.index, columns=["pred"], dtype=float)

building_numbers = X["building_number"].unique()
```

```
for bnum in building_numbers:
    x_building = X[X['building_number'] == bnum].copy()
    y_building = Y[Y['building_number'] == bnum]['power_consumption'].copy()
    xt_building = test_X[test_X['building_number'] == bnum].copy()

    # 현재 건물의 유형을 확인하고, max_depth_dict에서 해당 유형에 맞는 max_depth 값 추출
    # --> 건물 유형별로 모델의 복잡도를 다르게 설정하기 위함
    current_building_type = X[X['building_number'] == bnum]['building_type'].iloc[0]
    current_max_depth = max_depth_dict.get(current_building_type, 10)

    # 모델 학습에 필요없는 칼럼 제거
    drop_cols_building = ["building_type", "building_number"]
    x_building = x_building.drop(columns=drop_cols_building, errors='ignore')
    xt_building = xt_building.drop(columns=drop_cols_building, errors='ignore')

    xt_building = xt_building.reindex(columns=x_building.columns, fill_value=0)

    preds_valid_building = pd.Series(index=y_building.index, dtype=float)
    preds_test_building = []

    x_values_building = x_building.values
    y_values_building = y_building.values

    fold_scores_building = []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(x_values_building), 1):
        X_tr, X_va = x_values_building[tr_idx], x_values_building[va_idx]
        y_tr, y_va = y_values_building[tr_idx], y_values_building[va_idx]

        # 타겟 변수(y)의 분포를 정규화하기 위해 로그 변환을 적용
        y_tr_log = np.log(y_tr)
        y_va_log = np.log(y_va)

        model_building = XGBRegressor(
            learning_rate     = 0.05,
            n_estimators      = 5000,
            max_depth         = current_max_depth,
            subsample         = 0.7,
            colsample_bytree  = 0.5,
            min_child_weight  = 3,
            random_state      = RANDOM_SEED,
            objective         = weighted_mse(3),
            tree_method       = "gpu_hist",
            gpu_id            = 0,
            early_stopping_rounds = 100,
        )

        model_building.fit(
            X_tr, y_tr_log,
            eval_set=[(X_va, y_va_log)],
            eval_metric=custom_smape,
            verbose=False,
        )

        # 로그 변환된 예측값을 다시 원래 스케일로 복구
        va_pred = np.exp(model_building.predict(X_va))
        preds_valid_building.iloc[va_idx] = va_pred

        fold_smape = smape(y_va, va_pred)
        fold_scores_building.append(fold_smape)

        preds_test_building.append(np.exp(model_building.predict(xt_building.values)))

    # 현재 건물의 교차 검증 예측값을 pred_df_by_building에 할당
    pred_df_by_building.loc[preds_valid_building.index, "pred"] = preds_valid_building

    # 각 폴드에서 계산된 테스트 예측값들의 평균을 구해 최종 예측값으로 사용하고 answer_df_by_building에 저장
    answer_df_by_building.loc[xt_building.index, "answer"] = np.mean(preds_test_building, axis=0)

    print(f"Building number = {bnum} : XGB SMAPE = {np.mean(fold_scores_building):.4f}")

total_smape_by_building = smape(
    Y.sort_index()["power_consumption"].values,
    pred_df_by_building.sort_index()["pred"].values
)
print(f"Total SMAPE (by Building) = {total_smape_by_building:.4f}")
```
---

#### 1+2+3 앙상블

```
answer_df = pd.read_csv(f'{DATA_DIR}/answer_test_nosummer{RANDOM_SEED}.csv')
answer_df_by_building = pd.read_csv(f'{DATA_DIR}/answer_test_by_building_nosummer{RANDOM_SEED}.csv')
answer_df_global = pd.read_csv(f'{DATA_DIR}/answer_test_by_global_nosummer{RANDOM_SEED}.csv')

final_ensemble_test_pred = (
    answer_df.sort_index()["answer"].values * 0.25 +
    answer_df_by_building.sort_index()["answer"].values * 0.25 +
    answer_df_global.sort_index()["answer"].values * 0.5
)

final_ensemble_test_pred_fixed = [max(0, x) for x in final_ensemble_test_pred]
```
---
#### 4. 클러스터 기반 모델
- KMeans를 이용해 전력 소비 패턴이 유사한 건물들을 5개 그룹으로 군집화 후, 그룹별 모델링

```
 if cluster == True:
        # 각 건물의 요일 및 시간대별 평균 전력 소비량을 요약한 피벗 테이블 생성
        pivot_table = train.pivot_table(
            values='power_consumption',
            index='building_number',
            columns=['day_of_week', 'hour'],
            aggfunc='mean'
        ).fillna(0)

        pivot_table.columns = [f'dow_{dow}_hour_{hour}' for (dow, hour) in pivot_table.columns]

        k = 5
        kmeans = KMeans(n_clusters=k, random_state=2025, n_init=10)
        clusters = kmeans.fit_predict(pivot_table)

        building_info = building_info.set_index('building_number')
        building_info['cluster'] = pd.Series(clusters, index=pivot_table.index)
        building_info = building_info.reset_index()

        train = pd.merge(train, building_info[['building_number', 'cluster']], on='building_number', how='left')
        test = pd.merge(test, building_info[['building_number', 'cluster']], on='building_number', how='left')

        # 각 클러스터에 속한 건물의 개수를 계산
        cluster_counts = building_info['cluster'].value_counts().sort_index()
        print("Cluster-wise building count:")
        print(cluster_counts)

        total_buildings = building_info['building_number'].nunique()
        print("\nTotal number of buildings:", total_buildings)
```

```
# 이전에 클러스터링으로 생성된 고유한 클러스터 번호
cluster_list = sorted(train["cluster"].unique())

# 각 클러스터에 대해 모델의 복잡도 조절
max_depth_dict_cluster = {
    0: 10,
    1: 8,
    2: 10,
    3: 8,
    4: 10
}

answer_df_cluster = pd.DataFrame(index=test_X.index, columns=["answer"], dtype=float)
pred_df_cluster   = pd.DataFrame(index=X.index,         columns=["pred"],    dtype=float)

kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=RANDOM_SEED)

for cluster_num in cluster_list:
    x  = X[X['cluster'] == cluster_num].copy()
    y  = Y[Y['cluster'] == cluster_num]['power_consumption'].copy()
    xt = test_X[test_X['cluster'] == cluster_num].copy()

    x  = pd.get_dummies(x,  columns=["building_number"], drop_first=False)
    xt = pd.get_dummies(xt, columns=["building_number"], drop_first=False)

    xt = xt.reindex(columns=x.columns, fill_value=0)

    drop_cols = ["cluster"]
    x  = x.drop(columns=drop_cols)
    xt = xt.drop(columns=drop_cols)

    preds_valid = pd.Series(index=y.index, dtype=float)
    preds_test  = []

    x_values = x.values
    y_values = y.values

    fold_scores = []
    for fold, (tr_idx, va_idx) in enumerate(kf.split(x_values), 1):
        X_tr, X_va = x_values[tr_idx], x_values[va_idx]
        y_tr, y_va = y_values[tr_idx], y_values[va_idx]

        y_tr_log = np.log(y_tr)
        y_va_log = np.log(y_va)

        model = XGBRegressor(
            learning_rate      = 0.05,
            n_estimators      = 5000,
            max_depth          = max_depth_dict_cluster[cluster_num],
            subsample          = 0.7,
            colsample_bytree  = 0.5,
            min_child_weight  = 3,
            random_state      = RANDOM_SEED,
            objective          = weighted_mse(3),
            tree_method        = "gpu_hist",
            gpu_id            = 0,
            early_stopping_rounds = 100,
        )

        model.fit(
            X_tr, y_tr_log,
            eval_set=[(X_va, y_va_log)],
            eval_metric=custom_smape,
            verbose=False,
        )

        va_pred = np.exp(model.predict(X_va))
        preds_valid.iloc[va_idx] = va_pred

        fold_smape = smape(y_va, va_pred)
        fold_scores.append(fold_smape)

        preds_test.append(np.exp(model.predict(xt.values)))

    pred_df_cluster.loc[preds_valid.index, "pred"] = preds_valid
    answer_df_cluster.loc[xt.index, "answer"] = np.mean(preds_test, axis=0)

    print(f"Building Cluster = {cluster_num} : XGB SMAPE = {np.mean(fold_scores):.4f}")

total_smape = smape(
    Y.sort_index()["power_consumption"].values,
    pred_df_cluster.sort_index()["pred"].values
)
print(f"Total SMAPE = {total_smape:.4f}")
```

#### 5. Summer Feature 추가

```
def Preprocessing(summer = False, cluster = False):
...

위 모델링 동일하게 진행

## **배울 점**

#### 1. 컬럼명 변환
- 한국어로 되어있는 컬럼명을 영어로 변환하여 모델링 과정의 효율성과 편의성을 높임

```
 train = train.rename(columns={
        '건물번호': 'building_number',
        '일시': 'date_time',
        '기온(°C)': 'temperature',
        '강수량(mm)': 'rainfall',
        '풍속(m/s)': 'windspeed',
        '습도(%)': 'humidity',
        '일조(hr)': 'sunshine',
        '일사(MJ/m2)': 'solar_radiation',
        '전력소비량(kWh)': 'power_consumption'
    })
```



#### 2. 다양한 파생변수 생성

- 도메인 지식 활용

  : 단순한 원시 데이터(온도, 습도)를 넘어, 냉방 부하(CDH)나 체감 온도(WCT)와 같은 도메인 지식이 담긴 복합 지표를 만들었다는 점

  : 주말과 특정 공휴일을 별도의 변수로 만들어서 모델이 휴일의 전력 소비량 감소 패턴을 학습할 수 있게 한 것

- 시간 데이터 처리

  : 주기성 변환: 시간, 요일, 월 같은 주기적인 데이터를 sin과 cos 함수로 변환
  - 시간의 연속성과 주기성을 모두 표현하여, 모델이 12월 31일과 1월 1일이 시간적으로는 가깝다는 순환적 관계를 이해하도록 도움

- 함수 생성으로 효율적인 파생변수 생성

  : calculate_day_values 함수처럼 재사용 가능한 코드를 만들어 여러 파생 변수를 효율적으로 생성

- 실험적 접근

  : summer_cos와 summer_sin처럼 특정 기간(여름)에만 해당하는 변수를 만들어 모델이 해당 시기의 패턴을 더 잘 포착하도록 유도

#### 3. 여러 종류의 모델링

- 단순히 하나의 모델을 학습시키는 것이 아니라, 데이터를 여러 관점에서 나누고 각각의 모델을 학습시킨 뒤 최종 결과를 결합하는 앙상블 전략을 사용

--> 이는 모델의 성능과 안정성을 극대화하기 위한 중요한 접근법

- 모델링 최적화를 위한 기법
  1. 하이브리드 모델링: '여름 특성 추가'와 같이 특정 시기에만 적용되는 피처를 실험적으로 추가하는 하이브리드 접근법을 사용
  
  --> 이는 특정 계절적 패턴을 모델이 더 민감하게 포착하도록 도움

  2. 로그 변환: 타겟 변수인 전력 소비량에 np.log를 적용하여 데이터 분포를 정규화
  
  --> 이는 예측값이 0에 가까워질 때 발생하는 오차를 줄이고, 모델이 안정적으로 학습하도록 돕는 중요한 전처리 과정

  3. 파이프라인 구축: Preprocessing 함수를 통해 전처리 과정을 모듈화하여 재사용성을 높임
  
  --> 이는 다양한 모델링 실험(summer 또는 cluster 피처 추가/제거)을 용이하게 함

- 최종 앙상블 및 가중치 부여
  1. 모델 결합: 최종 예측값은 '건물 유형별 모델', '건물별 모델', '글로벌 모델'의 예측값에 각각 가중치를 부여하여 결합

  2. 가중치 설정: 코드에서 0.25, 0.25, 0.5와 같이 특정 모델에 더 높은 가중치를 부여
  
  --> 이는 전체 모델이 가장 일반적인 패턴을 잘 학습했다고 판단했기 때문일 수 있음. 이처럼 각 모델의 특성을 고려하여 가중치를 조정하는 것은 앙상블 성능을 최적화하는 핵심적인 과정!

결론적으로, **다양한 수준의 세분화된 모델링 전략**을 이용하였음